<a href="https://colab.research.google.com/github/santoshkumarbvp/recommendor-system/blob/master/Content_Based_RecSys_and_data_imported_from_google_drive.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials
# Authenticate and create the PyDrive client.
auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)

In [0]:
link = 'https://drive.google.com/open?id=1zFCZZ0MzKMSr1XXHBSS33cUEvAbT1FeA'

In [0]:
fluff, id = link.split('=')

In [0]:
import pandas as pd
import numpy as np
import io

In [0]:
downloaded = drive.CreateFile({'id':id}) 
downloaded.GetContentFile('metadata_with_overview.csv')  
df = pd.read_csv('metadata_with_overview.csv')

In [11]:
df.head()

,Unnamed: 0,title,runtime,vote_average,vote_count,year,genre,Id,Overview
0,0,Toy Story,81.0,7.7,5415.0,1995,"['Animation', 'Comedy', 'Family']",862,"Led by Woody, Andy's toys live happily in his ..."
1,1,Jumanji,104.0,6.9,2413.0,1995,"['Adventure', 'Fantasy', 'Family']",8844,When siblings Judy and Peter discover an encha...
2,2,Grumpier Old Men,101.0,6.5,92.0,1995,"['Romance', 'Comedy']",15602,A family wedding reignites the ancient feud be...
3,3,Waiting to Exhale,127.0,6.1,34.0,1995,"['Comedy', 'Drama', 'Romance']",31357,"Cheated on, mistreated and stepped on, the wom..."
4,4,Father of the Bride Part II,106.0,5.7,173.0,1995,['Comedy'],11862,Just when George Banks has recovered from his ...


In [0]:
from sklearn.feature_extraction.text import TfidfVectorizer

In [0]:
tfidf = TfidfVectorizer(stop_words='english')

In [14]:
df['Overview'].isnull().sum()

954

In [0]:
df['Overview'] = df['Overview'].fillna('')

In [16]:
df['Overview'].isnull().sum()

0

In [0]:
tfidf_matrix = tfidf.fit_transform(df['Overview'])

In [0]:
from sklearn.metrics.pairwise import linear_kernel

In [0]:
cosine_similarity = linear_kernel(tfidf_matrix,tfidf_matrix)

In [20]:
cosine_similarity.shape

(45466, 45466)

In [0]:
indices = pd.Series(df.index,index = df['title'])

In [22]:
indices.shape

(45466,)

In [0]:
def content_based_recsys(movie_liked, cosine_similarity = cosine_similarity, indices = indices, df=df ):
  # finding the index number for respective liked movie
  idx = indices[movie_liked]
  # finding cosine similarity list for all movies corresponding to liked movie (idx)
  cosine_sim_score_ml = list(enumerate(cosine_similarity[idx]))
  # now sort all scores in cosine_sim_score_ml
  cosine_sim_score_ml = sorted(cosine_sim_score_ml,key= lambda x : x[1],reverse=True)
  # Get the top 10 scores 
  cosine_sim_score_ml = cosine_sim_score_ml[1:11]
  # get the movie indices
  movie_indices = [x[0] for x in cosine_sim_score_ml]
  # return the top 10 most similar movie
  return df['title'].iloc[movie_indices]



In [24]:
content_based_recsys('African Cats')

27933              Massaï, les guerriers de la pluie
42829                                           Prey
34682    How the Lion Cub and the Turtle Sang a Song
359                                    The Lion King
25654                                 Fearless Fagan
26922                                    Vampire Dog
31208                                     White Lion
9353                                The Lion King 1½
6094                                       Born Free
30772                                    The Couples
Name: title, dtype: object